In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

### 데이터 전처리

In [5]:
df = pd.read_csv('data/hub_data.csv', index_col=0)

In [6]:
df

,class,conversation
0,0,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,0,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,3,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,1,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,1,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...
4945,4,"자기야, 나 이번에 학원 선생님이랑 친해졌는데 자꾸 나를 수산물 시장으로 불러내서 ..."
4946,4,"하, 자기야. 나 이번에 계약한 중고차 또 자동차 검사받았어.\n헉, 저번 주에 정..."
4947,4,천장으로 올라간 고양이가 내려오지를 않아. 아무리 불러도 꿈쩍도 안 해서 정말 불안...
4948,4,자기야 나 사고친 것 같아서 너무 불안하고 당혹스러워. 어제 회식자리에서 취해서 부...


### 모델 

In [7]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.model_selection import train_test_split

In [8]:
MODEL_NAME = "klue/bert-base"

In [9]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [10]:
encoded_inputs = tokenizer(
    list(df['conversation']),
    padding='max_length', # 또는 padding=True (배치 내 최대 길이에 맞춤)
    truncation=True,
    max_length=128,       # BERT 모델이 처리 가능한 최대 길이 고려 (klue/bert-base는 512)
    return_tensors='tf'
)

In [11]:
labels = tf.constant(df['class'].values)
unique_labels = np.unique(labels.numpy())
NUM_CLASSES = len(unique_labels)

In [12]:
num_samples = len(df)
indices = np.arange(num_samples)

train_indices, val_indices = train_test_split(
    indices,
    test_size=0.2,
    random_state=42,
    stratify=labels.numpy() # stratify에는 NumPy 배열 전달
)

In [13]:
train_inputs = {key: tf.gather(val, train_indices) for key, val in encoded_inputs.items()}
val_inputs = {key: tf.gather(val, val_indices) for key, val in encoded_inputs.items()}

# 레이블도 동일한 인덱스로 선택
train_labels = tf.gather(labels, train_indices)
val_labels = tf.gather(labels, val_indices)

In [14]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_inputs, train_labels))
train_dataset = train_dataset.shuffle(len(train_indices)).batch(16) # 셔플 및 배치

# 예시: 검증 데이터셋 생성
val_dataset = tf.data.Dataset.from_tensor_slices((val_inputs, val_labels))
val_dataset = val_dataset.batch(16) # 검증 데이터는 보통 셔플하지 않음

In [15]:
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_CLASSES, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

In [17]:
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [18]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  110617344 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3845      
Total params: 110,621,189
Trainable params: 110,621,189
Non-trainable params: 0
_________________________________________________________________


In [26]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping_cb = EarlyStopping(
    monitor="val_accuracy", 
    min_delta=0.001, # the threshold that triggers the termination (acc should at least improve 0.001)
    patience=2)

model_checkpoint_cb = ModelCheckpoint(
    filepath='model_weight.h5', # 저장 경로
    monitor='val_loss',           # 관찰 대상: 검증 손실
    save_best_only=True,          # 최고의 모델만 저장
    save_weights_only=True,       # 가중치만 저장
    mode='min',                   # val_loss는 낮을수록 좋으므로 'min'
    verbose=1                     # 저장 시 로그 출력
)

In [27]:
NUM_EPOCHS = 50
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=NUM_EPOCHS, # 충분한 에폭 수 지정 (조기 종료가 관리)
    callbacks=[early_stopping_cb, model_checkpoint_cb] # 정의된 콜백 전달
)

Epoch 1/50
248/248 [==============================] - 4976s 20s/step - loss: 0.4666 - accuracy: 0.8391 - val_loss: 0.2850 - val_accuracy: 0.9010

Epoch 00001: val_loss improved from inf to 0.28501, saving model to model_weight
Epoch 2/50
214/248 [========================>.....] - ETA: 10:36 - loss: 0.1710 - accuracy: 0.9436

KeyboardInterrupt: 